## Color Classifier Project

This document presents a simulated approach for the development of a `color classifier`. In a real-life scenario a machine learning framework such as PyTorch or TensorFlow would be used.

I preten to use `labeled` training data, which is unicolor images, categorized into directories named after the respective colors `(red, green, blue)`. 

The labeled data would be loaded using the data loading routine in PyTorch et al and split into training and validation dataset - usually an 80/20 split is used for model training.

A valid approach for the classifier could be a `Convolutional Neural Network (CNN)`. In this project a much more simple approach is utilized to show the case. The `RGB_classifier`can be found in the `Jupyter Notebook`.

**Important Note**: When ML modeling it is of critical importance to separate training and testing datasets to prevent data leakage and overfitting.


**Let's assume that the classifier has been constructed during the described training.**
The code in the Jupyter notebook shows several scenarios where images are classified into their respective color or tagged as tagged as unclassified.

For sake of simplicty multiple 3x3 pixel images, with an 8-bit color depth have been created via the `create_image` function. Every pixel is represented by an RGB color value triplet e.g. [0, 0, 255] for blue.   

In the test scenario two images are expected to be classified as blue and green, while the third is tagged as unclassified. This scenario simulated the application of the trained model in a real-world scenario.


In [3]:
# Import the Image class from the Python Imaging Library and the numpy package
from PIL import Image
import numpy as np

def create_image(width, height, rgb_values, image_name='color_image.png'):
    """
    Create and save an image based on provided RGB values.
    
    Args:
    width (int): The width of the image in pixels.
    height (int): The height of the image in pixels.
    rgb_values (list of list of tuples): A matrix where each tuple represents RGB values.
    image_name (str, optional): The name of the file to save the image as. Defaults to 'color_image.png'.
    
    Returns:
    Image: An Image object created by PIL, representing the image.
    """
    # Initialize an array of zeros with the shape (height, width, 3) for RGB color channels.
    # np.uint8 is an 8-bit unsigned integer type, allowing values between 0 and 255.
    array = np.zeros((height, width, 3), dtype=np.uint8)

    # Set RGB values for each pixel.
    for y in range(height):
        for x in range(width):
            array[y, x] = rgb_values[y][x]  # Assign the RGB values from the input matrix to the array

    # Create an Image object from the numpy array in RGB mode.
    img = Image.fromarray(array, 'RGB')
    img.save(image_name)  # Save the image to a file with the specified image name
    return img


#
# Parameters for a BLUE example image
#
width = 3
height = 3
rgb_values = [
    [(0, 0, 255), (0, 0, 255), (0, 0, 255)], 
    [(0, 0, 255), (0, 0, 255), (0, 0, 255)],  
    [(0, 0, 255), (255, 0, 0),   (255, 0, 0)]  # Introduce a pattern with red pixels
]
image_name = 'almost_blue.png'
image = create_image(width, height, rgb_values, image_name=image_name)

# image.show()  # Uncomment to display the image
# display(image)  # Uncomment to display the image in a notebook environment


# Create GREEN example image:
width = 3
height = 3
rgb_values = [
    [(0, 255, 0), (0, 255, 0), (0, 255, 0)], 
    [(0, 255, 0), (0, 255, 0), (0, 255, 0)],  
    [(0, 255, 0), (255, 0, 0),   (255, 0, 0)]  
]
image_name = 'almost_green.png'
image = create_image(width, height, rgb_values, image_name=image_name)

# Create Mix example image:
width = 3
height = 3
rgb_values = [
    [(255, 255, 255), (255, 255, 255), (255, 255, 255)], 
    [(255, 255, 255), (255, 255, 255), (255, 255, 255)],  
    [(255, 255, 255), (255, 255, 255), (255, 255, 255)]  
]
image_name = 'almost_mix.png'
image = create_image(width, height, rgb_values, image_name=image_name)


# image.show() 
# display(image)

## Classifier
The basic classifier function below takes an 8bit RGB image, loads it into a numpy array, calculates the mean values of all RGB values per pixel and returns the value and the classification based on the conditions outlined below.

In [4]:
def RGB_classifier(image_path):
    # Import necessary libraries
    from PIL import Image
    import numpy as np

    # Open the image file from the given path and ensure it is in RGB format
    # This step is necessary to handle different image formats uniformly
    image = Image.open(image_path)
    image = image.convert('RGB')
    
    # Convert the RGB image into a 3D NumPy array
    # The array structure will be (height, width, RGB channels)
    pixels = np.array(image)

    # Calculate the mean (average) RGB values across all pixels
    # np.mean calculates the mean along the specified axes (0 for height, 1 for width)
    # The result is a 1D array with three elements, each representing the mean of R, G, and B channels
    mean_colors = np.mean(pixels, axis=(0, 1))
    
    # Print the mean colors to see the average RGB values
    # print(mean_colors)
    
    # Determine the dominant RGB color based on the highest mean value
    # Compare each color's mean value and decide which one is the highest
    # This establishes which color is dominant in the image
    dominant_color = 'Unclassified'
    if mean_colors[0] > mean_colors[1] and mean_colors[0] > mean_colors[2]:
        dominant_color = 'Red'
    elif mean_colors[1] > mean_colors[0] and mean_colors[1] > mean_colors[2]:
        dominant_color = 'Green'
    elif mean_colors[2] > mean_colors[0] and mean_colors[2] > mean_colors[1]:
        dominant_color = 'Blue'
    
    # Return the dominant color and the mean RGB values as a tuple
    # This can be useful for further analysis or display purposes
    return dominant_color, mean_colors


## Testing the classifier
We have a blue and a green image and expect the results to be accordingly. To confirm the classifier works reasonably well create one image the is not RG or B but a mix that would be considered outside the valid scope. We expect this example to be taged as Unclassified.

In [5]:
color, means = RGB_classifier("almost_blue.png")
print("Dominant Color:", color)
print("Mean RGB Values:", means)

print("---------------------------------------------------------------------")

color, means = RGB_classifier("almost_green.png")
print("Dominant Color:", color)
print("Mean RGB Values:", means)

print("---------------------------------------------------------------------")

color, means = RGB_classifier("almost_mix.png")
print("Dominant Color:", color)
print("Mean RGB Values:", means)

Dominant Color: Blue
Mean RGB Values: [ 56.66666667   0.         198.33333333]
---------------------------------------------------------------------
Dominant Color: Green
Mean RGB Values: [ 56.66666667 198.33333333   0.        ]
---------------------------------------------------------------------
Dominant Color: Unclassified
Mean RGB Values: [255. 255. 255.]
